In [2]:
import json
import glob
import os
import numpy

In [28]:
def read_json_files(path_to_file):
    with open(path_to_file) as p:
        return json.load(p)
        

def giant_list(json_files):
    return [read_json_files(f) for f in json_files]

def parse_json(json_file):
    if len(json_file['people']) > 0:
        return json_file['people'][0].get('pose_keypoints_2d', [])
    
    return []

def format_XY(XY):
    X = [xy[::3] for xy in XY]
    Y = [xy[1::3] for xy in XY]
    
    return numpy.array(X), numpy.array(Y)

def normalize_input_data(input_data):
    #print(input_data)
    max_d, min_d = numpy.max(input_data), numpy.min(input_data)
    #print(max_d, min_d)
    return (input_data-min_d) / (max_d-min_d)

def preprocessing(file_path, cut_off=75):
    print(file_path)
    origin_json = giant_list(sorted(glob.glob(file_path+"/*.json"), key=os.path.getmtime))
    XY = [parse_json(j) for j in origin_json]
    XY = [xy for xy in XY if xy][:75]
    
    X, Y = format_XY(XY)
    print(X.shape, Y.shape)
    
    X = normalize_input_data(X)
    Y = normalize_input_data(Y)
    return [X, Y]
    
    
def preprocessiongDir(dir_path):
    path = glob.glob(os.path.join(dir_path, '*'))
    samples = [numpy.array(preprocessing(f)).reshape(-1) for f in path]
    print(len(samples))
    return numpy.array(samples)
    
    

In [31]:
train_data = preprocessiongDir('../input_data/train/')
train_target = numpy.array([1] * 10)

../input_data/train/MyPlayground-59-of-67
(75, 25) (75, 25)
../input_data/train/12-38
(75, 25) (75, 25)
../input_data/train/12-39
(75, 25) (75, 25)
../input_data/train/MyPlayground-52-of-67
(75, 25) (75, 25)
../input_data/train/0-13-of-19
(75, 25) (75, 25)
../input_data/train/0-15-of-19
(75, 25) (75, 25)
../input_data/train/0-10-of-19
(75, 25) (75, 25)
../input_data/train/test-2-of-9.mp4
(75, 25) (75, 25)
../input_data/train/0-6-of-19
(75, 25) (75, 25)
../input_data/train/0-14-of-19
(75, 25) (75, 25)
10


In [41]:
tensor_test = preprocessiongDir('../input_data/test/')
test_target = numpy.array([1, 1])

../input_data/test/test-7-of-9.mp4
(75, 25) (75, 25)
../input_data/test/MyPlayground-58-of-67
(75, 25) (75, 25)
2


In [42]:
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

#MLP model structre
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=3750))
model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()


model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_target, 
          epochs=30,batch_size=16,verbose=1)

score = model.evaluate(tensor_test, test_target)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 256)               960256    
_________________________________________________________________
dense_14 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_15 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 17        
Total params: 995,233
Trainable params: 995,233
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
10/10 [==============================] - 0s 32ms/step - loss: 1.3358 - acc: 0.0000e+00
Epoch 2/30
10/10 [==============================] - 0s 794us/step - loss: 8.1182e-06 - acc: 1.0000
Epoch 3/30
10/10 [==============================] - 0s 662us/step - los

In [43]:
print(score)

[1.192093321833454e-07, 1.0]
